## CPSC 340 Lecture 22: stochastic gradient descent

This notebook is for the in-class activities. It assumes you have already watched the [associated video](https://www.youtube.com/watch?v=lmqV5Z5HZzc&list=PLWmXHcz_53Q02ZLeAxigki1JZFfCO6M-b&index=22).

<font color='red'>**REMINDER TO START RECORDING**</font>

Also, reminder to enable screen sharing for Participants.

## Pre-class music



1. 911 by Lady Gaga
2. 

## Admin

- a5 posted

## Video chapters

- kernel trick recap
- stochastic gradient motivation
- SGD vs. GD
- minimizing averages
- per-example gradients
- visualizing SGD: 1 parameter
- decreasing step sizes
- stochastic average gradient
- visualizing SGD: 2 parameters
- summary

![](img/L21/image.png)

## Old exam questions

#### 2016W2 final

Name an advantage and a disadvantage of using an SVM with the RBF kernel instead of a linear kernel.

#### 2018W1 final

![](img/L21/matching2_top.png)

![](img/L21/matching2_bottom.png)

## Demo of the "other" normal equations and the kernel trick

Consider the randomly generated dataset below:

In [41]:
import numpy as np
n = 20
d = 3
p = 2
np.random.seed(1)
X = np.random.randint(0,10,size=(n,d))
X

array([[5, 8, 9],
       [5, 0, 0],
       [1, 7, 6],
       [9, 2, 4],
       [5, 2, 4],
       [2, 4, 7],
       [7, 9, 1],
       [7, 0, 6],
       [9, 9, 7],
       [6, 9, 1],
       [0, 1, 8],
       [8, 3, 9],
       [8, 7, 3],
       [6, 5, 1],
       [9, 3, 4],
       [8, 1, 4],
       [0, 3, 9],
       [2, 0, 4],
       [9, 2, 7],
       [7, 9, 8]])

In [24]:
y = np.random.randint(-5, 5, n)
y

array([ 1,  4, -2,  2,  2, -1,  0,  4, -2,  1,  3, -5, -3,  2,  2,  4,  2,
       -2, -5,  3])

We decide to use a degree 2 polynomial basis to fit this dataset, using L2 regularization with $\lambda=1$. 

We then want to make a prediction on the test vector:

In [25]:
Xtest = np.array([[-1, 0, 1]])

Code this up 3 ways:

1. Using the "regular" normal equations
2. Using the "other" normal equations without the kernel trick
3. Using the "other" normal equations with the kernel trick

and show that you get the same answers all 3 ways.

In [26]:
from sklearn.preprocessing import PolynomialFeatures

#### Approach 1

In [42]:
poly = PolynomialFeatures(degree=p)
Z = poly.fit_transform(X)

Running time of the above step: 

- We have $O(d^p)$ features we're making. Let's call this $O(k)$ for now to keep it general and we can plug $k=d^p$ back in at the end.
- We need to transform each of the $n$ training examples. 
- For each training example, for each new feature, we potentially need to look at all the old features.
- $O(ndk)$

In [35]:
w = np.linalg.solve(Z.T@Z + np.eye(poly.n_output_features_), Z.T@y)
w

array([-1.325003  ,  1.62053602, -1.77771719,  0.1154218 , -0.02284804,
       -0.0384322 , -0.22760804,  0.13310881,  0.11101713,  0.04284918])

Running time of the above step:

- Multiply $Z^TZ$, meaning an $k \times n$ multiplied by $n \times k$ which is $O(nk^2)$
- Then the solving is $O(k^3)$ so we have $O(nk^2 + k^3)$ which we've seen in an earlier lecture.

In [32]:
poly.transform(Xtest)@w

array([-2.58250803])

Running time of the above step:

- $O(k)$
- So the total is $O(ndk+nk^2+k^3)$.

#### Approach 2

In [34]:
w_other = Z.T @ np.linalg.solve(Z@Z.T + np.eye(n), y)
w_other

array([-1.325003  ,  1.62053602, -1.77771719,  0.1154218 , -0.02284804,
       -0.0384322 , -0.22760804,  0.13310881,  0.11101713,  0.04284918])

Running time of the above step:

- Multiply $ZZ^T$, meaning an $n \times k$ multiplied by $k \times n$ which is $O(n^2k)$
- Then the solving is $O(n^3)$ so we have $O(n^2k + n^3)$ which we've seen in an earlier lecture.

We can see we get the same $w$ so the prediction will also be the same.

#### Approach 3

$K(a, b) = (1+a^Tb)^p$

In [43]:
K = (1 + np.sum(X[None] * X[:,None], axis=-1))**p

In [45]:
K.shape

(20, 20)

- $K$ is $n \times n$ so it costs $O(n^2d)$

In [51]:
v = np.linalg.solve(K + np.eye(n), y)

In [55]:
v.shape

(20,)

- Costs $n^3$ to solve the $n \times n$ system.

In [52]:
Ktest = (1 + np.sum(X[None] * Xtest[:,None], axis=-1))**p

In [53]:
Ktest.shape

(1, 20)

In [54]:
Ktest @ v

array([-2.8150272])

And we get the same prediction!

- Total cost $O(n^2d + n^3)$ which is independent of $k$.

Summary of running times:

- Regular approach $O(ndk+nk^2+k^3)$ 


## Question

What is the **space complexity** of each of the above 3 approaches?

## Question

What is the **prediction time complexity** (for one example) for each of the above 3 approaches?